In [ ]:
import os, sys
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ShuffleSplit, cross_val_score
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import tensorflow as tf
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, f1_score
from sklearn.metrics import r2_score
from sklearn import linear_model
from scipy.stats import linregress
from scipy.stats import spearmanr
from scipy.stats import pearsonr

from sklearn.base import  BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.signal import butter, filtfilt, lfilter, welch, lfilter_zi, stft
import pandas as pd

from sklearn.model_selection import LeaveOneOut
import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [ ]:
!unzip /content/drive/MyDrive/CX_Regression/datosFrank/conectividad_physionet_sujetos.zip

Archive:  /content/drive/MyDrive/CX_Regression/datosFrank/conectividad_physionet_sujetos.zip
Written using ZipTricks 4.8.2
 extracting: Data_s65_phy.mat        
 extracting: Data_s29_phy.mat        
 extracting: Data_s26_phy.mat        
 extracting: Data_s89_phy.mat        
 extracting: Data_s105_phy.mat       
 extracting: Data_s97_phy.mat        
 extracting: Data_s56_phy.mat        
 extracting: Data_s58_phy.mat        
 extracting: Data_s49_phy.mat        
 extracting: Data_s54_phy.mat        
 extracting: Data_s33_phy.mat        
 extracting: Data_s47_phy.mat        
 extracting: Data_s57_phy.mat        
 extracting: Data_s88_phy.mat        
 extracting: Data_s98_phy.mat        
 extracting: Data_s94_phy.mat        
 extracting: Data_s95_phy.mat        
 extracting: Data_s85_phy.mat        
 extracting: Data_s44_phy.mat        
 extracting: Data_s27_phy.mat        
 extracting: Data_s76_phy.mat        
 extracting: Data_s72_phy.mat        
 extracting: Data_s52_phy.mat        
 ex

In [ ]:
! pip install bctpy
! pip install mne==0.19
#%% descargar código de funciones  para motor imagery
#%% download data
FILEID = "1DOm75xoSPUbnFfcJBjVggPS0hzNEoWCG"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O MIfunctions.py && rm -rf /tmp/cookies.txt
#%% download data


     |████████████████████████████████| 112kB 4.2MB/s 
     |████████████████████████████████| 6.4MB 4.2MB/s 
--2021-02-03 21:46:41--  https://docs.google.com/uc?export=download&confirm=&id=1DOm75xoSPUbnFfcJBjVggPS0hzNEoWCG
Resolving docs.google.com (docs.google.com)... 108.177.125.139, 108.177.125.113, 108.177.125.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.125.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/mgburnp1fih7djnhhcgu4geuda4qo3ba/tcomfc45ja26q13jbc9792u55od2qhd1/1612388775000/06932581800131476027/10479963849793151253Z/1DOm75xoSPUbnFfcJBjVggPS0hzNEoWCG?e=download [following]
--2021-02-03 21:46:42--  https://doc-0s-c0-docs.googleusercontent.com/docs/securesc/mgburnp1fih7djnhhcgu4geuda4qo3ba/tcomfc45ja26q13jbc9792u55od2qhd1/1612388775000/06932581800131476027/10479963849793151253Z/1DOm75xoSPUbnFfcJBjVggPS0hzNEoWCG?e=download
Resolving doc-0s-c0-docs.goo

In [ ]:
load.keys()

dict_keys(['acc_subj', 'indx'])

In [ ]:
##cargar datos csp e indx
load = pickle.load( open( "/content/drive/MyDrive/CX_Regression/codigos/physionet/targets_physionet/target_phy_2seg_5time_2frec.p", "rb" ) )
indx = load['indx']
indx2=[int(indx[i][1::]) for i in range(len(indx))]
target = np.asarray(load['acc_subj'])
target[np.argsort(indx2)]


array([0.5       , 0.6       , 0.54444444, 0.51111111, 0.47777778,
       0.51111111, 0.87777778, 0.58888889, 0.56666667, 0.55555556,
       0.57777778, 0.61111111, 0.51111111, 0.51111111, 0.71111111,
       0.43333333, 0.57777778, 0.6       , 0.61111111, 0.46666667,
       0.57777778, 0.58888889, 0.52222222, 0.45555556, 0.55555556,
       0.48888889, 0.5       , 0.42222222, 0.97777778, 0.43333333,
       0.68888889, 0.66666667, 0.5       , 0.64444444, 0.62222222,
       0.5       , 0.54444444, 0.48888889, 0.5       , 0.66666667,
       0.52222222, 0.78888889, 0.56666667, 0.52222222, 0.58888889,
       0.51111111, 0.5       , 0.72222222, 0.6       , 0.57777778,
       0.44444444, 0.52222222, 0.57777778, 0.8       , 0.57777778,
       0.75555556, 0.47777778, 0.48888889, 0.46666667, 0.57777778,
       0.5       , 0.74444444, 0.61111111, 0.41111111, 0.47777778,
       0.55555556, 0.5       , 0.62222222, 0.57777778, 0.68888889,
       0.57777778, 0.75555556, 0.61111111, 0.6       , 0.54444

In [ ]:
array([2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
conectividad = loadmat('/content/Data_s1_phy.mat')['data_laplacian']
print(conectividad.shape)
###### 
#crear tensor (sujeto x canales x tiempo)
xtensor = []
subs_order = indx2
for i in range(1,106):
  try:
    s=loadmat('/content/Data_s'+str(i)+'_phy.mat')['data_laplacian']
    s=s[0:6350].T
    xtensor+=[s]
  except:
    print('s'+str(i+1)+'_phy.mat')
xtensor= np.asarray(xtensor)
xtensor.shape

In [ ]:
conectividad = loadmat('/content/Data_s1_phy.mat')['data_laplacian']
print(conectividad.shape)
###### 
#crear tensor (sujeto x canales x tiempo)
xtensor = []
subs_order = indx2
for i in subs_order:
  if i == 104:
    print('s'+str(i)+'_phy.mat')    
  else:
    try:
      s=loadmat('/content/Data_s'+str(i)+'_phy.mat')['data_laplacian']
      s=s[0:6350].T
      xtensor+=[s]
    except:
      print('s'+str(i)+'_phy.mat')
xtensor= np.asarray(xtensor)
xtensor.shape

(6561, 64)
s109_phy.mat
s104_phy.mat
s108_phy.mat
s107_phy.mat
s106_phy.mat


(101, 64, 6350)